# CSP (Cash-Secured Put) Strategy - Phase 1: Data Layer

This notebook establishes the data infrastructure for the options selling strategy.

**Phase 1 Objectives:**
1. Configuration management
2. Alpaca API client setup
3. VIX data fetching (via yfinance)
4. Equity historical data fetching
5. Options chain fetching
6. Data validation and testing

## 1. Setup & Configuration

In [ ]:
# Install required packages (run once)
# !pip install alpaca-py yfinance pandas numpy python-dotenv

In [5]:
import os
from dataclasses import dataclass, field
from typing import Dict, List, Optional, Tuple
from datetime import datetime, date, timedelta
from enum import Enum
import pandas as pd
import numpy as np
import yfinance as yf
from dotenv import load_dotenv

# Alpaca imports
from alpaca.data.historical import StockHistoricalDataClient
from alpaca.data.requests import StockBarsRequest
from alpaca.data.timeframe import TimeFrame
from alpaca.trading.client import TradingClient
from alpaca.trading.requests import GetOptionContractsRequest
from alpaca.trading.enums import AssetStatus

# Load environment variables
load_dotenv()

print("Imports successful!")

Imports successful!


In [67]:
@dataclass
class StrategyConfig:
    """
    Central configuration for the CSP strategy.
    All parameters in one place for easy tuning.
    """
    # ==================== UNIVERSE & CAPITAL ====================
    ticker_universe: List[str] = field(default_factory=lambda: 
    [
        'AAPL', 'MSFT', 'GOOG'
    ])
    num_tickers: int = 10  # Max positions for diversification
    starting_cash: float = 50_000
    
    # ==================== VIX REGIME RULES ====================
    # (vix_lower, vix_upper): deployment_multiplier
    vix_deployment_rules: Dict[Tuple[float, float], float] = field(default_factory=lambda: {
        (0, 15): 1.0,       # VIX < 15: deploy 100%
        (15, 20): 0.8,      # 15 <= VIX < 20: deploy 80%
        (20, 25): 0.2,      # 20 <= VIX < 25: deploy 20%
        (25, float('inf')): 0.0  # VIX >= 25: deploy 0%
    })
    
    # ==================== EQUITY FILTER PARAMS ====================
    sma_periods: List[int] = field(default_factory=lambda: [8, 20, 50])
    rsi_period: int = 14
    rsi_lower: int = 30
    rsi_upper: int = 70
    bb_period: int = 50
    bb_std: float = 1.0
    sma_trend_lookback: int = 3  # Days to confirm SMA(50) uptrend
    history_days: int = 60  # Days of price history to fetch
    
    # ==================== OPTIONS FILTER PARAMS ====================
    min_daily_return: float = 0.15  # 0.15% daily yield on strike (premium/dte/strike)
    max_strike_pct: float = 0.98      # Strike <= 90% of stock price
    min_strike_pct: float = 0.85      # Strike <= 90% of stock price
    delta_min: float = 0.15
    delta_max: float = 0.40
    max_dte: int = 10
    min_dte: int = 1  # Avoid 0 DTE
    
    # ==================== RISK MANAGEMENT ====================
    delta_stop_multiplier: float = 2.0   # Exit if delta >= 2x entry
    stock_drop_stop_pct: float = 0.05    # Exit if stock drops 5%
    vix_spike_multiplier: float = 1.15   # Exit if VIX >= 1.15x entry
    early_exit_buffer: float = 0.15      # Exit if 15% ahead of decay schedule
    
    # ==================== OPERATIONAL ====================
    poll_interval_seconds: int = 60
    paper_trading: bool = True  # Safety first!
    
    def get_deployment_multiplier(self, vix: float) -> float:
        """Get cash deployment multiplier based on current VIX."""
        for (lower, upper), multiplier in self.vix_deployment_rules.items():
            if lower <= vix < upper:
                return multiplier
        return 0.0  # Default to no deployment if VIX out of range
    
    def get_deployable_cash(self, vix: float) -> float:
        """Calculate deployable cash based on VIX regime."""
        return self.starting_cash * self.get_deployment_multiplier(vix)


# Initialize config
config = StrategyConfig()

# Test VIX deployment rules
print("VIX Deployment Rules Test:")
for test_vix in [12, 17, 22, 30]:
    deployable = config.get_deployable_cash(test_vix)
    print(f"  VIX={test_vix}: Deploy ${deployable:,.0f} ({config.get_deployment_multiplier(test_vix):.0%})")

VIX Deployment Rules Test:
  VIX=12: Deploy $50,000 (100%)
  VIX=17: Deploy $40,000 (80%)
  VIX=22: Deploy $10,000 (20%)
  VIX=30: Deploy $0 (0%)


## 2. Alpaca Client Setup

In [68]:
class AlpacaClientManager:
    """
    Manages Alpaca API clients for data and trading.
    Handles authentication and provides unified access.
    """
    
    def __init__(self, paper: bool = True):
        """
        Initialize Alpaca clients.
        
        Args:
            paper: If True, use paper trading credentials
        """
        self.paper = paper
        
        # Get credentials from environment
        if paper:
            self.api_key = os.getenv('ALPACA_API_KEY')
            self.secret_key = os.getenv('ALPACA_SECRET_KEY')
        
        if not self.api_key or not self.secret_key:
            raise ValueError(
                "Alpaca credentials not found. Set environment variables:\n"
                "  ALPACA_API_KEY and ALPACA_SECRET_KEY (or ALPACA_PAPER_* variants)"
            )
        
        # Initialize clients
        self._data_client = None
        self._trading_client = None
    
    @property
    def data_client(self) -> StockHistoricalDataClient:
        """Lazy initialization of data client."""
        if self._data_client is None:
            self._data_client = StockHistoricalDataClient(
                api_key=self.api_key,
                secret_key=self.secret_key
            )
        return self._data_client
    
    @property
    def trading_client(self) -> TradingClient:
        """Lazy initialization of trading client."""
        if self._trading_client is None:
            self._trading_client = TradingClient(
                api_key=self.api_key,
                secret_key=self.secret_key,
                paper=self.paper
            )
        return self._trading_client
    
    def get_account_info(self) -> dict:
        """Get account information."""
        account = self.trading_client.get_account()
        return {
            'cash': float(account.cash),
            'buying_power': float(account.buying_power),
            'portfolio_value': float(account.portfolio_value),
            'status': account.status,
            'trading_blocked': account.trading_blocked,
            'options_trading_level': getattr(account, 'options_trading_level', None)
        }


# Test client initialization (will fail without credentials, that's expected)
try:
    alpaca = AlpacaClientManager(paper=config.paper_trading)
    account = alpaca.get_account_info()
    print("✓ Alpaca connection successful!")
    print(f"  Account status: {account['status']}")
    print(f"  Cash available: ${account['cash']:,.2f}")
    print(f"  Options level: {account['options_trading_level']}")
except ValueError as e:
    print(f"⚠ Credentials not configured: {e}")
    print("\nTo configure, create a .env file or set environment variables:")
    print("  ALPACA_API_KEY=your_api_key")
    print("  ALPACA_SECRET_KEY=your_secret_key")
    alpaca = None
except Exception as e:
    print(f"⚠ Connection error: {e}")
    alpaca = None

✓ Alpaca connection successful!
  Account status: AccountStatus.ACTIVE
  Cash available: $50,000.00
  Options level: 3


## 3. VIX Data Fetcher

Since Alpaca doesn't provide VIX directly, we use yfinance for `^VIX`.

In [69]:
class VixDataFetcher:
    """
    Fetches VIX data from Yahoo Finance.
    Provides current VIX and historical data for analysis.
    """
    
    SYMBOL = "^VIX"
    
    def __init__(self):
        self._ticker = yf.Ticker(self.SYMBOL)
        self._cache = {}
        self._cache_time = None
        self._cache_ttl = timedelta(minutes=1)
    
    def get_current_vix(self) -> float:
        """
        Get the current/latest VIX value.
        Uses last trading day's close when market is closed.
        
        Returns:
            Current VIX value as float
        """
        if (self._cache_time and 
            datetime.now() - self._cache_time < self._cache_ttl and
            'current' in self._cache):
            return self._cache['current']
        
        try:
            # Use 5d history - more reliable than 1d on weekends
            daily = self._ticker.history(period='5d')
            if daily.empty:
                raise RuntimeError("No VIX data available")
            
            vix = float(daily['Close'].iloc[-1])
            
            self._cache['current'] = vix
            self._cache_time = datetime.now()
            
            return vix
            
        except Exception as e:
            raise RuntimeError(f"Failed to fetch VIX data: {e}")
    
    def get_vix_history(self, days: int = 30) -> pd.DataFrame:
        """
        Get historical VIX OHLC data.
        
        Args:
            days: Number of days of history
            
        Returns:
            DataFrame with Open, High, Low, Close columns
        """
        history = self._ticker.history(period=f'{days}d')
        return history[['Open', 'High', 'Low', 'Close']]
    
    def get_last_session(self) -> dict:
        """
        Get the most recent completed trading session's data.
        
        Returns:
            Dict with session_date, open, high, low, close
        """
        history = self._ticker.history(period='5d')
        if history.empty:
            raise RuntimeError("No VIX history available")
        
        last_row = history.iloc[-1]
        session_date = history.index[-1]
        
        return {
            'session_date': session_date.date() if hasattr(session_date, 'date') else session_date,
            'open': float(last_row['Open']),
            'high': float(last_row['High']),
            'low': float(last_row['Low']),
            'close': float(last_row['Close']),
        }
    
    def get_session_reference_vix(self) -> Tuple[date, float]:
        """
        Get the reference VIX for stop-loss calculations.
        
        During market hours: that day's open
        Outside market hours: last trading day's open (for next session planning)
        
        Returns:
            Tuple of (session_date, reference_vix)
        """
        session = self.get_last_session()
        return session['session_date'], session['open']
    
    def check_vix_stop_loss(
        self, 
        reference_vix: float, 
        multiplier: float = config.vix_spike_multiplier
    ) -> dict:
        """
        Check if VIX stop-loss condition is triggered.
        Condition: current_vix >= reference_vix * multiplier
        
        Args:
            reference_vix: The VIX value to compare against (entry or session open)
            multiplier: Stop-loss multiplier (default 1.15 = 15% spike)
            
        Returns:
            Dict with triggered (bool), current_vix, threshold, reason
        """
        current_vix = self.get_current_vix()
        threshold = reference_vix * multiplier
        triggered = current_vix >= threshold
        
        return {
            'triggered': triggered,
            'current_vix': current_vix,
            'reference_vix': reference_vix,
            'threshold': threshold,
            'pct_change': (current_vix / reference_vix - 1) * 100,
            'reason': f"VIX {current_vix:.2f} >= {threshold:.2f}" if triggered else ""
        }


# Test VIX fetcher
vix_fetcher = VixDataFetcher()

try:
    current_vix = vix_fetcher.get_current_vix()
    last_session = vix_fetcher.get_last_session()
    
    print(f"✓ VIX Data Retrieved")
    print(f"\n  Last Trading Session: {last_session['session_date']}")
    print(f"    Open:  {last_session['open']:.2f}")
    print(f"    High:  {last_session['high']:.2f}")
    print(f"    Low:   {last_session['low']:.2f}")
    print(f"    Close: {last_session['close']:.2f}")
    
    print(f"\n  Current VIX: {current_vix:.2f}")
    print(f"  Deployment: {config.get_deployment_multiplier(current_vix):.0%}")
    print(f"  Deployable Cash: ${config.get_deployable_cash(current_vix):,.0f}")
    
    # Test stop-loss check using session open as reference
    stop_loss_check = vix_fetcher.check_vix_stop_loss(
        reference_vix=last_session['open'],
        multiplier=config.vix_spike_multiplier
    )
    
    print(f"\n  VIX Stop-Loss Check (vs session open):")
    print(f"    Reference: {stop_loss_check['reference_vix']:.2f}")
    print(f"    Threshold: {stop_loss_check['threshold']:.2f} ({config.vix_spike_multiplier:.0%})")
    print(f"    Current:   {stop_loss_check['current_vix']:.2f} ({stop_loss_check['pct_change']:+.1f}%)")
    print(f"    Triggered: {'🚨 YES - EXIT ALL' if stop_loss_check['triggered'] else '✓ No'}")
    
    # Show recent history
    vix_history = vix_fetcher.get_vix_history(10)
    print(f"\n  Last 5 sessions:")
    print(f"    {'Date':<12} {'Open':>8} {'Close':>8}")
    print(f"    {'-'*12} {'-'*8} {'-'*8}")
    for dt, row in vix_history.tail(5).iterrows():
        print(f"    {dt.strftime('%Y-%m-%d'):<12} {row['Open']:>8.2f} {row['Close']:>8.2f}")
        
except Exception as e:
    print(f"⚠ VIX fetch error: {e}")
    import traceback
    traceback.print_exc()

✓ VIX Data Retrieved

  Last Trading Session: 2026-01-30
    Open:  18.72
    High:  19.27
    Low:   16.67
    Close: 17.44

  Current VIX: 17.44
  Deployment: 80%
  Deployable Cash: $40,000

  VIX Stop-Loss Check (vs session open):
    Reference: 18.72
    Threshold: 21.53 (115%)
    Current:   17.44 (-6.8%)
    Triggered: ✓ No

  Last 5 sessions:
    Date             Open    Close
    ------------ -------- --------
    2026-01-26      16.90    16.15
    2026-01-27      16.02    16.35
    2026-01-28      16.09    16.35
    2026-01-29      16.04    16.88
    2026-01-30      18.72    17.44


## 4. Equity Historical Data Fetcher

In [70]:
class EquityDataFetcher:
    """
    Fetches equity price data from Alpaca.
    Provides historical bars and current prices.
    """
    
    def __init__(self, alpaca_manager: AlpacaClientManager):
        self.client = alpaca_manager.data_client
    
    def get_close_history(
        self, 
        symbols: List[str], 
        days: int = 60
    ) -> Dict[str, pd.Series]:
        """
        Get closing price history for multiple symbols.
        
        Args:
            symbols: List of ticker symbols
            days: Number of trading days of history
            
        Returns:
            Dict mapping symbol -> pd.Series of close prices
        """
        # Calculate date range (add buffer for weekends/holidays)
        end_date = datetime.now()
        start_date = end_date - timedelta(days=int(days * 1.5))  # Buffer for non-trading days
        
        request = StockBarsRequest(
            symbol_or_symbols=symbols,
            timeframe=TimeFrame.Day,
            start=start_date,
            end=end_date
        )
        
        bars = self.client.get_stock_bars(request)
        
        # Process into dict of Series
        result = {}
        for symbol in symbols:
            if symbol in bars.data:
                symbol_bars = bars.data[symbol]
                closes = pd.Series(
                    [bar.close for bar in symbol_bars],
                    index=[bar.timestamp for bar in symbol_bars],
                    name=symbol
                )
                # Take last N days
                result[symbol] = closes.tail(days)
            else:
                print(f"  Warning: No data for {symbol}")
        
        return result
    
    def get_current_price(self, symbol: str) -> float:
        """
        Get the most recent price for a symbol.
        
        Args:
            symbol: Ticker symbol
            
        Returns:
            Latest close price
        """
        history = self.get_close_history([symbol], days=5)
        if symbol in history and len(history[symbol]) > 0:
            return float(history[symbol].iloc[-1])
        raise ValueError(f"No price data for {symbol}")
    
    def get_current_prices(self, symbols: List[str]) -> Dict[str, float]:
        """
        Get current prices for multiple symbols efficiently.
        
        Args:
            symbols: List of ticker symbols
            
        Returns:
            Dict mapping symbol -> current price
        """
        history = self.get_close_history(symbols, days=5)
        return {
            symbol: float(prices.iloc[-1]) 
            for symbol, prices in history.items() 
            if len(prices) > 0
        }


# Test equity data fetcher
if alpaca:
    equity_fetcher = EquityDataFetcher(alpaca)
    
    # Test with subset of universe
    test_symbols = config.ticker_universe[:3]  # First 3 symbols
    
    try:
        close_history = equity_fetcher.get_close_history(test_symbols, days=50)
        
        print(f"✓ Equity Data Retrieved for {len(close_history)} symbols")
        for symbol, prices in close_history.items():
            print(f"\n  {symbol}:")
            print(f"    Data points: {len(prices)}")
            print(f"    Date range: {prices.index[0].strftime('%Y-%m-%d')} to {prices.index[-1].strftime('%Y-%m-%d')}")
            print(f"    Current price: ${prices.iloc[-1]:.2f}")
            print(f"    50-day range: ${prices.min():.2f} - ${prices.max():.2f}")
            
    except Exception as e:
        print(f"⚠ Equity fetch error: {e}")
else:
    print("⚠ Skipping equity test - Alpaca not configured")

✓ Equity Data Retrieved for 3 symbols

  AAPL:
    Data points: 49
    Date range: 2025-11-19 to 2026-01-30
    Current price: $259.48
    50-day range: $246.70 - $286.19

  MSFT:
    Data points: 49
    Date range: 2025-11-19 to 2026-01-30
    Current price: $430.29
    50-day range: $430.29 - $492.02

  GOOG:
    Data points: 49
    Date range: 2025-11-19 to 2026-01-30
    Current price: $338.53
    50-day range: $289.98 - $338.66


## 5. Options Chain Fetcher

This is the most complex data component. Alpaca's options API requires specific handling.

In [71]:
import numpy as np
from py_vollib.black_scholes.implied_volatility import implied_volatility
from py_vollib.black_scholes.greeks.analytical import delta as bs_delta

# Risk-free rate (can pull from config or treasury API later)
RISK_FREE_RATE = 0.04


class GreeksCalculator:
    """
    Calculates IV and Delta using Black-Scholes via py_vollib.
    Falls back gracefully when calculation fails.
    """
    
    def __init__(self, risk_free_rate: float = RISK_FREE_RATE):
        self.r = risk_free_rate
    
    def compute_iv(
        self,
        option_price: float,
        stock_price: float,
        strike: float,
        dte: int,
        option_type: str = 'put'
    ) -> Optional[float]:
        """
        Compute implied volatility from option price.
        
        Args:
            option_price: Mid price of the option
            stock_price: Current underlying price
            strike: Strike price
            dte: Days to expiration
            option_type: 'put' or 'call'
            
        Returns:
            IV as decimal (e.g., 0.25 for 25%) or None if calculation fails
        """
        t = dte / 365.0
        flag = 'p' if option_type == 'put' else 'c'
        
        # Validate inputs
        if not all([
            np.isfinite(option_price),
            np.isfinite(stock_price),
            np.isfinite(strike),
            t > 0,
            option_price > 0,
            stock_price > 0,
            strike > 0
        ]):
            return None
        
        try:
            iv = implied_volatility(option_price, stock_price, strike, t, self.r, flag)
            return iv if np.isfinite(iv) and iv > 0 else None
        except Exception:
            return None
    
    def compute_delta(
        self,
        stock_price: float,
        strike: float,
        dte: int,
        iv: float,
        option_type: str = 'put'
    ) -> Optional[float]:
        """
        Compute delta from IV.
        
        Args:
            stock_price: Current underlying price
            strike: Strike price
            dte: Days to expiration
            iv: Implied volatility as decimal
            option_type: 'put' or 'call'
            
        Returns:
            Delta (negative for puts) or None if calculation fails
        """
        if iv is None or not np.isfinite(iv) or iv <= 0:
            return None
        
        t = dte / 365.0
        flag = 'p' if option_type == 'put' else 'c'
        
        if t <= 0:
            return None
        
        try:
            d = bs_delta(flag, stock_price, strike, t, self.r, iv)
            return d if np.isfinite(d) else None
        except Exception:
            return None
    
    def compute_greeks(
        self,
        option_price: float,
        stock_price: float,
        strike: float,
        dte: int,
        option_type: str = 'put'
    ) -> dict:
        """
        Compute both IV and delta in one call.
        
        Returns:
            Dict with 'iv' and 'delta' keys (values may be None)
        """
        iv = self.compute_iv(option_price, stock_price, strike, dte, option_type)
        delta = self.compute_delta(stock_price, strike, dte, iv, option_type) if iv else None
        
        return {
            'iv': iv,
            'delta': delta,
            'delta_abs': abs(delta) if delta else None
        }


# Test the calculator
greeks_calc = GreeksCalculator()

# Test with one of the contracts that had Greeks from Alpaca
# AAPL260206P00225000: strike=225, dte=5, mid=0.15, alpaca_delta=-0.0214, alpaca_iv=0.6077
test_result = greeks_calc.compute_greeks(
    option_price=0.15,
    stock_price=259.48,  # approximate from your data
    strike=225.0,
    dte=5,
    option_type='put'
)

print("Greeks Calculator Test:")
print(f"  Calculated IV: {test_result['iv']:.4f}" if test_result['iv'] else "  IV: Failed")
print(f"  Calculated Delta: {test_result['delta']:.4f}" if test_result['delta'] else "  Delta: Failed")
print(f"\nAlpaca provided:")
print(f"  Alpaca IV: 0.6077")
print(f"  Alpaca Delta: -0.0214")


Greeks Calculator Test:
  Calculated IV: 0.6136
  Calculated Delta: -0.0212

Alpaca provided:
  Alpaca IV: 0.6077
  Alpaca Delta: -0.0214


In [ ]:
from alpaca.trading.requests import GetOptionContractsRequest
from alpaca.trading.enums import ContractType, AssetStatus
from alpaca.data.historical.option import OptionHistoricalDataClient
from alpaca.data.requests import OptionLatestQuoteRequest, OptionSnapshotRequest

# daily return meaning secured_amount = 100*underlying_share_price
# premium/DTE -- daily cash / cash_outlayed = daily_return_as_ptct
# daily_return = (premium/DTE) 

@dataclass
class OptionContract:
    """
    Represents a single option contract with relevant data.
    """
    symbol: str
    underlying: str
    contract_type: str
    strike: float
    expiration: date
    dte: int
    bid: float
    ask: float
    mid: float
    stock_price: float  # ADD THIS - needed for return calculations
    delta: Optional[float] = None
    gamma: Optional[float] = None
    theta: Optional[float] = None
    vega: Optional[float] = None
    implied_volatility: Optional[float] = None
    open_interest: Optional[int] = None
    volume: Optional[int] = None
    
    @property
    def premium(self) -> float:
        """Premium received when selling (use bid price)."""
        return self.bid
    
    @property
    def premium_per_day(self) -> float:
        """Daily premium decay if held to expiration."""
        if self.dte <= 0:
            return 0.0
        return self.premium / self.dte
    
    @property
    def collateral_required(self) -> float:
        """Cash required to secure 1 contract."""
        return self.strike * 100
    
    @property
    def cost_basis(self) -> float:
        """Cost basis = stock price * 100 (exposure value)."""
        return self.stock_price * 100
    
    @property
    def daily_return_on_collateral(self) -> float:
        """Daily yield as % of collateral (strike-based)."""
        if self.strike <= 0 or self.dte <= 0:
            return 0.0
        return self.premium_per_day / self.strike
    
    @property
    def daily_return_on_cost_basis(self) -> float:
        """Daily yield as % of cost basis (stock price-based)."""
        if self.stock_price <= 0 or self.dte <= 0:
            return 0.0
        return self.premium_per_day / self.stock_price
    
    @property
    def delta_abs(self) -> Optional[float]:
        """Absolute value of delta for filtering."""
        return abs(self.delta) if self.delta else None

class OptionsDataFetcher:
    """
    Fetches options chain data from Alpaca.
    Handles contract discovery and quote retrieval.
    """
    
    def __init__(self, alpaca_manager: AlpacaClientManager):
        self.trading_client = alpaca_manager.trading_client
        self.data_client = OptionHistoricalDataClient(
            api_key=alpaca_manager.api_key,
            secret_key=alpaca_manager.secret_key
        )
    
    def get_option_contracts(
        self,
        underlying: str,
        contract_type: str = 'put',
        min_dte: int = config.min_dte,
        max_dte: int = config.max_dte,
        min_strike: Optional[float] = None,
        max_strike: Optional[float] = None
    ) -> List[dict]:
        """
        Get available option contracts for an underlying.
        
        Args:
            underlying: Ticker symbol
            contract_type: 'put' or 'call'
            min_dte: Minimum days to expiration
            max_dte: Maximum days to expiration
            min_strike: Minimum strike price
            max_strike: Maximum strike price
            
        Returns:
            List of contract dictionaries
        """
        # Calculate expiration date range
        today = date.today()
        min_expiry = today + timedelta(days=min_dte)
        max_expiry = today + timedelta(days=max_dte)
        
        request_params = {
            'underlying_symbols': [underlying],
            'status': AssetStatus.ACTIVE,
            'type': ContractType.PUT if contract_type == 'put' else ContractType.CALL,
            'expiration_date_gte': min_expiry,
            'expiration_date_lte': max_expiry,
        }
        
        if min_strike is not None:
            request_params['strike_price_gte'] = str(min_strike)
        if max_strike is not None:
            request_params['strike_price_lte'] = str(max_strike)
        
        request = GetOptionContractsRequest(**request_params)
        
        response = self.trading_client.get_option_contracts(request)
        
        # Convert to list of dicts
        contracts = []
        if response.option_contracts:
            for contract in response.option_contracts:
                contracts.append({
                    'symbol': contract.symbol,
                    'underlying': contract.underlying_symbol,
                    'strike': float(contract.strike_price),
                    'expiration': contract.expiration_date,
                    'contract_type': contract_type,
                })
        
        return contracts
    
    def get_option_quotes(
        self, 
        option_symbols: List[str]
    ) -> Dict[str, dict]:
        """
        Get current quotes for option contracts.
        
        Args:
            option_symbols: List of OCC option symbols
            
        Returns:
            Dict mapping symbol -> quote data
        """
        if not option_symbols:
            return {}
        
        # Alpaca has limits on batch size, chunk if needed
        chunk_size = 100
        all_quotes = {}
        
        for i in range(0, len(option_symbols), chunk_size):
            chunk = option_symbols[i:i + chunk_size]
            
            try:
                request = OptionLatestQuoteRequest(symbol_or_symbols=chunk)
                quotes = self.data_client.get_option_latest_quote(request)
                
                for symbol, quote in quotes.items():
                    all_quotes[symbol] = {
                        'bid': float(quote.bid_price) if quote.bid_price else 0.0,
                        'ask': float(quote.ask_price) if quote.ask_price else 0.0,
                        'bid_size': quote.bid_size,
                        'ask_size': quote.ask_size,
                    }
            except Exception as e:
                print(f"  Warning: Quote fetch error for chunk: {e}")
        
        return all_quotes
    
    def get_option_snapshots(
        self,
        option_symbols: List[str]
    ) -> Dict[str, dict]:
        """
        Get snapshots including Greeks for option contracts.
        
        Args:
            option_symbols: List of OCC option symbols
            
        Returns:
            Dict mapping symbol -> snapshot data with Greeks
        """
        if not option_symbols:
            return {}
        
        chunk_size = 100
        all_snapshots = {}
        
        for i in range(0, len(option_symbols), chunk_size):
            chunk = option_symbols[i:i + chunk_size]
            
            try:
                request = OptionSnapshotRequest(symbol_or_symbols=chunk)
                snapshots = self.data_client.get_option_snapshot(request)
                
                for symbol, snapshot in snapshots.items():
                    greeks = snapshot.greeks if snapshot.greeks else None
                    quote = snapshot.latest_quote if snapshot.latest_quote else None
                    
                    all_snapshots[symbol] = {
                        'bid': float(quote.bid_price) if quote and quote.bid_price else 0.0,
                        'ask': float(quote.ask_price) if quote and quote.ask_price else 0.0,
                        'delta': float(greeks.delta) if greeks and greeks.delta else None,
                        'gamma': float(greeks.gamma) if greeks and greeks.gamma else None,
                        'theta': float(greeks.theta) if greeks and greeks.theta else None,
                        'vega': float(greeks.vega) if greeks and greeks.vega else None,
                        'implied_volatility': float(snapshot.implied_volatility) if snapshot.implied_volatility else None,
                    }
            except Exception as e:
                print(f"  Warning: Snapshot fetch error for chunk: {e}")
        
        return all_snapshots
    
    def get_puts_chain(
        self,
        underlying: str,
        stock_price: float,
        config: StrategyConfig
    ) -> List[OptionContract]:
        """
        Get filtered put options chain with full data.
        """
        # Get contracts within strike range
        max_strike = stock_price * config.max_strike_pct
        min_strike = stock_price * 0.70

        contracts = self.get_option_contracts(
            underlying=underlying,
            contract_type='put',
            min_dte=config.min_dte,
            max_dte=config.max_dte,
            min_strike=min_strike,
            max_strike=max_strike
        )

        if not contracts:
            return []

        # Get snapshots with Greeks
        symbols = [c['symbol'] for c in contracts]
        snapshots = self.get_option_snapshots(symbols)

        # Build OptionContract objects
        today = date.today()
        result = []

        for contract in contracts:
            symbol = contract['symbol']
            snapshot = snapshots.get(symbol, {})

            bid = snapshot.get('bid', 0.0)
            ask = snapshot.get('ask', 0.0)

            # Skip if no bid (can't sell)
            if bid <= 0:
                continue

            dte = (contract['expiration'] - today).days

            option = OptionContract(
                symbol=symbol,
                underlying=underlying,
                contract_type='put',
                strike=contract['strike'],
                expiration=contract['expiration'],
                dte=dte,
                bid=bid,
                ask=ask,
                mid=(bid + ask) / 2,
                stock_price=stock_price,  
                delta=snapshot.get('delta'),
                gamma=snapshot.get('gamma'),
                theta=snapshot.get('theta'),
                vega=snapshot.get('vega'),
                implied_volatility=snapshot.get('implied_volatility'),
            )

            result.append(option)

        return result



# Test options data fetcher
if alpaca:
    options_fetcher = OptionsDataFetcher(alpaca)
    
    test_symbol = 'AAPL'
    
    try:
        # Get current price
        current_price = equity_fetcher.get_current_price(test_symbol)
        print(f"Testing options chain for {test_symbol} @ ${current_price:.2f}")
        
        # Get puts chain
        puts = options_fetcher.get_puts_chain(test_symbol, current_price, config)
        
        print(f"\n✓ Retrieved {len(puts)} put contracts")
        
        if puts:
            print(f"\nSample contracts (first 5):")
            for put in puts[:5]:
                delta_str = f"{abs(put.delta):.2f}" if put.delta else "N/A"
                print(f"  {put.symbol}")
                print(f"    Strike: ${put.strike:.2f} | DTE: {put.dte}")
                print(f"    Bid/Ask: ${put.bid:.2f}/${put.ask:.2f}")
                print(f"    Delta: {delta_str}")
                print(f"    Daily return: {put.daily_return_on_cost_basis:.4%}")
                print()
    except Exception as e:
        print(f"⚠ Options fetch error: {e}")
        import traceback
        traceback.print_exc()
else:
    print("⚠ Skipping options test - Alpaca not configured")

Testing options chain for AAPL @ $259.48

✓ Retrieved 83 put contracts

Sample contracts (first 5):
  AAPL260202P00220000
    Strike: $220.00 | DTE: 1
    Bid/Ask: $0.03/$0.04
    Delta: N/A
    Daily return: 0.0116%

  AAPL260202P00225000
    Strike: $225.00 | DTE: 1
    Bid/Ask: $0.03/$0.05
    Delta: N/A
    Daily return: 0.0116%

  AAPL260202P00230000
    Strike: $230.00 | DTE: 1
    Bid/Ask: $0.05/$0.08
    Delta: N/A
    Daily return: 0.0193%

  AAPL260202P00232500
    Strike: $232.50 | DTE: 1
    Bid/Ask: $0.03/$0.24
    Delta: N/A
    Daily return: 0.0116%

  AAPL260202P00235000
    Strike: $235.00 | DTE: 1
    Bid/Ask: $0.08/$0.17
    Delta: 0.02
    Daily return: 0.0308%



## 6. Unified Data Manager

Combines all data fetchers into a single interface for the strategy.

In [77]:
@dataclass
class MarketSnapshot:
    """
    Complete market state at a point in time.
    Used by strategy logic to make decisions.
    """
    timestamp: datetime
    vix_current: float
    vix_open: float
    deployable_cash: float
    equity_prices: Dict[str, float]  # symbol -> current price
    equity_history: Dict[str, pd.Series]  # symbol -> price history


class DataManager:
    """
    Unified data manager that combines all data sources.
    Provides a clean interface for strategy logic.
    """
    
    def __init__(
        self, 
        alpaca_manager: Optional[AlpacaClientManager],
        config: StrategyConfig
    ):
        self.config = config
        self.vix_fetcher = VixDataFetcher()
        
        if alpaca_manager:
            self.equity_fetcher = EquityDataFetcher(alpaca_manager)
            self.options_fetcher = OptionsDataFetcher(alpaca_manager)
        else:
            self.equity_fetcher = None
            self.options_fetcher = None
    
    def get_market_snapshot(self) -> MarketSnapshot:
        """
        Get complete market state for strategy decision-making.
        
        Returns:
            MarketSnapshot with all current market data
        """
        # VIX data
        vix_current = self.vix_fetcher.get_current_vix()
        vix_open = self.vix_fetcher.get_vix_open_today() or vix_current
        
        # Deployable cash based on VIX
        deployable_cash = self.config.get_deployable_cash(vix_current)
        
        # Equity data
        if self.equity_fetcher:
            equity_history = self.equity_fetcher.get_close_history(
                self.config.ticker_universe,
                days=self.config.history_days
            )
            equity_prices = {
                symbol: float(prices.iloc[-1])
                for symbol, prices in equity_history.items()
            }
        else:
            equity_history = {}
            equity_prices = {}
        
        return MarketSnapshot(
            timestamp=datetime.now(),
            vix_current=vix_current,
            vix_open=vix_open,
            deployable_cash=deployable_cash,
            equity_prices=equity_prices,
            equity_history=equity_history
        )
    
    def get_puts_for_symbol(
        self, 
        symbol: str, 
        stock_price: float
    ) -> List[OptionContract]:
        """
        Get filtered put options for a symbol.
        
        Args:
            symbol: Ticker symbol
            stock_price: Current stock price
            
        Returns:
            List of OptionContract objects
        """
        if not self.options_fetcher:
            raise RuntimeError("Options fetcher not configured")
        
        return self.options_fetcher.get_puts_chain(
            symbol, 
            stock_price, 
            self.config
        )
    
    def refresh_option_data(self, option_symbol: str) -> Optional[OptionContract]:
        """
        Refresh data for a single option (for position monitoring).
        
        Args:
            option_symbol: OCC option symbol
            
        Returns:
            Updated OptionContract or None if fetch fails
        """
        if not self.options_fetcher:
            return None
        
        snapshots = self.options_fetcher.get_option_snapshots([option_symbol])
        
        if option_symbol not in snapshots:
            return None
        
        snapshot = snapshots[option_symbol]
        
        # Parse symbol to extract details (OCC format)
        # Format: AAPL240119P00150000
        # We'd need to parse this - for now return partial data
        return snapshot  # Return raw snapshot for now


# Test unified data manager
data_manager = DataManager(alpaca, config)

try:
    print("Fetching market snapshot...")
    snapshot = data_manager.get_market_snapshot()
    
    print(f"\n✓ Market Snapshot @ {snapshot.timestamp.strftime('%Y-%m-%d %H:%M:%S')}")
    print(f"  VIX: {snapshot.vix_current:.2f} (Open: {snapshot.vix_open:.2f})")
    print(f"  Deployable Cash: ${snapshot.deployable_cash:,.0f}")
    print(f"  Equities tracked: {len(snapshot.equity_prices)}")
    
    if snapshot.equity_prices:
        print(f"\n  Sample prices:")
        for symbol, price in list(snapshot.equity_prices.items())[:5]:
            print(f"    {symbol}: ${price:.2f}")
            
except Exception as e:
    print(f"⚠ Snapshot error: {e}")
    import traceback
    traceback.print_exc()

Fetching market snapshot...
⚠ Snapshot error: 'VixDataFetcher' object has no attribute 'get_vix_open_today'


Traceback (most recent call last):
  File "/var/folders/6k/0v57cgbd2k37vp0lh44zby640000gn/T/ipykernel_66425/695374127.py", line 128, in <module>
    snapshot = data_manager.get_market_snapshot()
  File "/var/folders/6k/0v57cgbd2k37vp0lh44zby640000gn/T/ipykernel_66425/695374127.py", line 45, in get_market_snapshot
    vix_open = self.vix_fetcher.get_vix_open_today() or vix_current
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
AttributeError: 'VixDataFetcher' object has no attribute 'get_vix_open_today'


## 7. Data Validation & Summary

Verify all data components are working correctly.

In [ ]:
def run_data_layer_diagnostics(data_manager: DataManager) -> dict:
    """
    Run comprehensive diagnostics on the data layer.
    
    Returns:
        Dict with diagnostic results
    """
    results = {
        'vix': {'status': 'unknown', 'details': {}},
        'equity': {'status': 'unknown', 'details': {}},
        'options': {'status': 'unknown', 'details': {}},
    }
    
    # Test VIX
    print("Testing VIX data...")
    try:
        vix = data_manager.vix_fetcher.get_current_vix()
        vix_history = data_manager.vix_fetcher.get_vix_history(10)
        results['vix'] = {
            'status': 'ok',
            'details': {
                'current': vix,
                'history_points': len(vix_history),
                'history_range': f"{vix_history.min():.2f} - {vix_history.max():.2f}"
            }
        }
        print(f"  ✓ VIX OK: {vix:.2f}")
    except Exception as e:
        results['vix'] = {'status': 'error', 'details': {'error': str(e)}}
        print(f"  ✗ VIX Error: {e}")
    
    # Test Equity
    print("\nTesting equity data...")
    if data_manager.equity_fetcher:
        try:
            test_symbols = data_manager.config.ticker_universe[:3]
            history = data_manager.equity_fetcher.get_close_history(test_symbols, days=60)
            results['equity'] = {
                'status': 'ok',
                'details': {
                    'symbols_tested': len(test_symbols),
                    'symbols_returned': len(history),
                    'avg_data_points': np.mean([len(v) for v in history.values()])
                }
            }
            print(f"  ✓ Equity OK: {len(history)}/{len(test_symbols)} symbols")
        except Exception as e:
            results['equity'] = {'status': 'error', 'details': {'error': str(e)}}
            print(f"  ✗ Equity Error: {e}")
    else:
        results['equity'] = {'status': 'not_configured', 'details': {}}
        print("  ⚠ Equity: Not configured (no Alpaca credentials)")
    
    # Test Options
    print("\nTesting options data...")
    if data_manager.options_fetcher and results['equity']['status'] == 'ok':
        try:
            test_symbol = data_manager.config.ticker_universe[0]
            test_price = data_manager.equity_fetcher.get_current_price(test_symbol)
            puts = data_manager.get_puts_for_symbol(test_symbol, test_price)
            
            # Count puts with Greeks
            puts_with_delta = sum(1 for p in puts if p.delta is not None)
            
            results['options'] = {
                'status': 'ok',
                'details': {
                    'test_symbol': test_symbol,
                    'contracts_found': len(puts),
                    'contracts_with_greeks': puts_with_delta,
                }
            }
            print(f"  ✓ Options OK: {len(puts)} contracts for {test_symbol}")
            print(f"    {puts_with_delta}/{len(puts)} have Greeks")
        except Exception as e:
            results['options'] = {'status': 'error', 'details': {'error': str(e)}}
            print(f"  ✗ Options Error: {e}")
    else:
        results['options'] = {'status': 'not_configured', 'details': {}}
        print("  ⚠ Options: Not configured")
    
    # Summary
    print("\n" + "="*50)
    print("PHASE 1 DATA LAYER STATUS")
    print("="*50)
    
    all_ok = all(r['status'] == 'ok' for r in results.values())
    
    for component, result in results.items():
        status_icon = {
            'ok': '✓',
            'error': '✗',
            'not_configured': '⚠',
            'unknown': '?'
        }.get(result['status'], '?')
        print(f"  {status_icon} {component.upper()}: {result['status']}")
    
    if all_ok:
        print("\n🎉 All data components working! Ready for Phase 2.")
    else:
        print("\n⚠ Some components need attention before proceeding.")
    
    return results


# Run diagnostics
diagnostics = run_data_layer_diagnostics(data_manager)

## Next Steps: Phase 2 Preview

With the data layer complete, Phase 2 will implement:

1. **Technical Indicators** (SMA, RSI, Bollinger Bands)
2. **Equity Filter** (`equity_passes_filter()`)
3. **Options Filter** (`filter_and_rank_options()`)

The data structures and fetchers built here will feed directly into those components.

In [ ]:
# Preview: Save config and data manager for Phase 2
# These will be imported in the next notebook

print("Phase 1 Complete!")
print("\nObjects ready for Phase 2:")
print("  - config (StrategyConfig)")
print("  - data_manager (DataManager)")
print("  - OptionContract (dataclass)")
print("  - MarketSnapshot (dataclass)")